In [32]:
# [START import]
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np

In [33]:
# Import Distance Matrix 
df_distance = pd.read_excel('df_distance_matrix.xlsx', index_col=0)

# Transform to Numpy Array
distance_matrix = df_distance.to_numpy()

In [34]:
df_distance

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16
row0,0,548,776,696,582,274,502,194,308,194,536,502,388,354,468,776,662
row1,548,0,684,308,194,502,730,354,696,742,1084,594,480,674,1016,868,1210
row2,776,684,0,992,878,502,274,810,468,742,400,1278,1164,1130,788,1552,754
row3,696,308,992,0,114,650,878,502,844,890,1232,514,628,822,1164,560,1358
row4,582,194,878,114,0,536,764,388,730,776,1118,400,514,708,1050,674,1244
row5,274,502,502,650,536,0,228,308,194,240,582,776,662,628,514,1050,708
row6,502,730,274,878,764,228,0,536,194,468,354,1004,890,856,514,1278,480
row7,194,354,810,502,388,308,536,0,342,388,730,468,354,320,662,742,856
row8,308,696,468,844,730,194,194,342,0,274,388,810,696,662,320,1084,514
row9,194,742,742,890,776,240,468,388,274,0,342,536,422,388,274,810,468


In [35]:
# 创建一个用于存储数据的字典
data = {}

# 距离矩阵：每一对目的地之间的距离
data['distance_matrix'] = distance_matrix

# 打印目的地数量（距离矩阵的大小减一，因为包括了到自身的距离）
print("{:,} destinations".format(len(data['distance_matrix'][0]) - 1))


16 destinations


In [36]:
# 创建一个将距离矩阵转换成Excel文件的函数
def create_excel(data):
    # 获取距离矩阵的列数
    n_col = len(data['distance_matrix'][0])
    # 获取距离矩阵的行数
    n_row = len(data['distance_matrix'])

    # 为行创建标签，例如 'row0', 'row1', 'row2', ...
    list_row = ['row' + str(i) for i in range(n_row)]
    # 为列创建标签，例如 'col0', 'col1', 'col2', ...
    list_col = ['col' + str(i) for i in range(n_row)]

    # 将距离矩阵转换为NumPy数组
    matrix = np.array(data['distance_matrix'])
    # 创建一个DataFrame，使用行和列标签
    df = pd.DataFrame(data=matrix, index=list_row, columns=list_col)
    
    # 将DataFrame保存为Excel文件
    df.to_excel('df_distance_matrix.xlsx')


In [37]:
# 订单数量（以盒子为单位）

data['demands'] = [0, 2, 3, 2, 4, 2, 4, 8, 8, 4, 2, 5, 2, 4, 4, 2, 2]#no solution可能原因是超载了
# 这行代码定义了一个名为 demands 的列表，用于表示每个目的地的订单数量（用盒子数表示）。列表中的第一个元素（0）通常代表起始点或仓库，它的
# 订单数量为 0。

# 车辆容量（以盒子为单位）
data['vehicle_capacities'] = [15, 15, 15, 15]
#这里，vehicle_capacities 是一个列表，代表每辆车的最大载货量（用盒子数表示）。在这个示例中，有四辆车，每辆车的载货量都是 15 盒。

# 车队信息
# 车辆数量
data['num_vehicles'] = 4

#num_vehicles 定义了车辆的总数，在这个例子中为 4 辆。

# 仓库（起始点）的位置
data['depot'] = 0

data['start_locations'] = [0, 1, 2, 3]#设置4个不同的起始点


In [38]:
print(sum(data['demands']))

58


In [39]:
def distance_callback(from_index, to_index):#把or-tools的内部索引转换为我们能看懂的实际索引，另外能计算距离，这个代码把距离作为代价使用了
    """
    Returns the distance between the two nodes.
    计算两个节点之间的距离。
    """
    # 将OR-Tools路由模型中的索引转换为距离矩阵中的节点索引
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)  # 起始节点的实际索引
    to_node = manager.IndexToNode(to_index)      # 目标节点的实际索引

    # 返回这两个节点之间的距离
    return data['distance_matrix'][from_node][to_node]


In [40]:
def demand_callback(from_index):#接收or-tools的索引，并转换为节点，然后从节点中取数
    """
    返回指定节点的需求量。
    Returns the demand of the node.
    """
    # 将OR-Tools路由模型中的索引转换为需求列表中的节点索引
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)  # 获取实际的节点索引

    # 返回该节点的需求量
    return data['demands'][from_node]

#这个函数的目的是返回特定节点的需求量。它首先使用 manager.IndexToNode 方法将 OR-Tools 路由模型中的索引转换为我们的需求列表中对应的节点索引。
#然后，它返回 data['demands'] 列表中对应节点的需求量。这个函数在车辆路径问题（VRP）中非常重要，用于确保每辆车分配的货物量不超过其容量限制，
#并且所有客户的需求都能被满足。

在 OR-Tools 的车辆路径问题（VRP）模型中，确实存在两类主要的函数和设置：一类用于问题建模和约束设置（大多通过 routing 对象进行）
另一类用于控制搜索过程的参数（通过 search_parameters 定义）

In [41]:
# # 创建路由索引管理器。
# manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
#                                        data['num_vehicles'], data['depot'])
# # 这个管理器用于帮助处理路由问题的索引,入参是送货目的地的数量，车的数量以及仓库位置


# 设置每辆车的起始点，这里仍然可以指定
start_locations = data['start_locations']

# 对于终点，我们为每辆车设置为 None，这种方法代码跑不通，先不建议
# end_locations = [None] * data['num_vehicles']


manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], 
                                       start_locations,
                                        [data['depot']] * data['num_vehicles'])#可以设置起始点和终点

# 创建路由模型
routing = pywrapcp.RoutingModel(manager)
# 这个模型用于定义和求解路由问题

# 创建并注册路径回调函数。回调函数是指函数作为参数传递，在别的函数运行的函数
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
# 这个回调函数用于计算两节点间的距离，设定用于计算两点间距离的回调函数，后面这个框架会自己去调用，这里函数作为入参对象，并不执行，这种叫做回调函数
#它的目的是把内部索引转换为能理解的距离

# 定义每个弧（路径段）的成本。
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
# 设置成本，通常是距离或时间,计算不同节点间的函数，简而言之，这行代码的核心作用是告诉 OR-Tools，如何计算问题中任意两个节点间的行驶成本
#这对于求解车辆路径问题并找到成本最优的路线方案是至关重要的。这里成本是用距离来评估的，因为distance_callback函数

In [42]:
# 添加容量约束。这部分routing.是重点，这里修改才能针对定制化的服务去实现
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index,#只是告诉取容量方法，并不执行回调函数
    0,  # 容量弹性空间为0
    data['vehicle_capacities'],  # 车辆最大容量
    True,  # 开始累积为0
    'Capacity')
# 这部分代码用于处理车辆的载货容量限制

# 添加距离维度，单车最大的

distance_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.AddDimension(
    distance_callback_index,
    0,  # slack max
    2000,  # 车辆的最大行驶距离
    True,  # start cumul to zero
    'Distance'
)

True

In [43]:
# 设置初始解决方案启发式。
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# 使用最便宜的路径段来构建初始解,相当于用最快的速度找一个局部最优解

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
# 设置本地搜索策略,找一个全局最优解

search_parameters.time_limit.FromSeconds(1)
# 设置搜索的时间限制，搜索时间局限为1秒

In [44]:
# 解决问题。
solution = routing.SolveWithParameters(search_parameters)#这里才会真正执行计算，之前的回调函数（如取数，计算距离）都在这在运行,使用routing来执行
# 使用之前设置的参数来找到问题的解


In [45]:
# 检查是否有解决方案,有解决方案的话，打印这个解决方案的所有信息
if solution:
    total_distance = 0  # 所有路线的总距离
    total_load = 0      # 所有路线的总货物量

    # 遍历每辆车
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)#获取每辆车的起始节点索引。
        plan_output = 'Route for AGV {}:\n'.format(vehicle_id)  # 车辆路线的输出字符串
        route_distance = 0  # 单个车辆的行驶距离
        route_load = 0      # 单个车辆的货物量

        # 遍历车辆的路线
        while not routing.IsEnd(index):#while not routing.IsEnd(index): 遍历直到路线结束。IsEnd 检查是否到达了车辆的终点。
            node_index = manager.IndexToNode(index)#将内部索引转换为节点编号
            route_load += data['demands'][node_index]  # 累加当前节点的货物量
            plan_output += ' {0} Parcels({1}) -> '.format(node_index, route_load)  # 添加节点信息到路线输出
            previous_index = index  # 存储当前节点索引
            index = solution.Value(routing.NextVar(index))  # 移动到下一个节点
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)  # 累加从当前节点到下一个节点的距离

        # 添加路线的结束信息
        plan_output += ' {0} Parcels({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {} (m)\n'.format(route_distance)
        plan_output += 'Parcels Delivered: {} (parcels)\n'.format(route_load)
        print(plan_output)  # 打印车辆路线信息

        # 累加到总距离和总货物量
        total_distance += route_distance
        total_load += route_load

    # 打印所有路线的总距离和总货物量
    print('Total distance of all routes: {:,} (m)'.format(total_distance))
    print('Parcels Delivered: {:,}/{:,}'.format(total_load, sum(data['demands'])))

else:
    print('No Solution')  # 如果没有找到解决方案


Route for AGV 0:
 0 Parcels(0) ->  8 Parcels(8) ->  6 Parcels(12) ->  5 Parcels(14) ->  0 Parcels(14)
Distance of the route: 1004 (m)
Parcels Delivered: 14 (parcels)

Route for AGV 1:
 1 Parcels(2) ->  4 Parcels(6) ->  7 Parcels(14) ->  0 Parcels(14)
Distance of the route: 776 (m)
Parcels Delivered: 14 (parcels)

Route for AGV 2:
 2 Parcels(3) ->  10 Parcels(5) ->  16 Parcels(7) ->  14 Parcels(11) ->  9 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1416 (m)
Parcels Delivered: 15 (parcels)

Route for AGV 3:
 3 Parcels(2) ->  15 Parcels(4) ->  11 Parcels(9) ->  12 Parcels(11) ->  13 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1496 (m)
Parcels Delivered: 15 (parcels)

Total distance of all routes: 4,692 (m)
Parcels Delivered: 58/58


# 待融入代码

# 订单优先级分割线

In [14]:
# 示例：添加时间窗口约束
time_callback_index = routing.RegisterTransitCallback(time_callback)
routing.AddDimension(
    time_callback_index,
    30,  # 时间窗口的缓冲（slack）
    30,  # 最大时间
    False,  # 不要求起始累积时间为零
    "Time"
)
time_dimension = routing.GetDimensionOrDie("Time")
for location_idx, time_window in enumerate(time_windows):
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])


NameError: name 'time_callback' is not defined

In [ ]:
# 示例：修改成本函数以包括优先级
def distance_callback_with_priority(from_index, to_index):
    # 基础距离
    distance = base_distance_callback(from_index, to_index)
    # 增加额外的成本，如果违反优先级
    additional_cost = calculate_priority_cost(from_index, to_index)
    return distance + additional_cost


# 设置起点和终点或者只设置起点

In [ ]:
# 每辆车的起始点
starts = [start_index1, start_index2, ...]

# 每辆车的结束点
ends = [end_index1, end_index2, ...]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], starts, ends)

In [ ]:
# 假设每辆车的起始点是确定的
starts = [start_index1, start_index2, ...]  # 起始点列表
ends = [None] * data['num_vehicles']  # 终点设置为 None

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], starts, ends)


# 设置最大距离

In [ ]:
# 首先定义距离回调函数
def distance_callback(from_index, to_index):
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

distance_callback_index = routing.RegisterTransitCallback(distance_callback)

# 添加距离维度，单车最大的
routing.AddDimension(
    distance_callback_index,
    0,  # slack max
    3000,  # 车辆的最大行驶距离
    True,  # start cumul to zero
    'Distance'
)
distance_dimension = routing.GetDimensionOrDie('Distance')

# 设置每辆车的最大行驶距离
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    distance_dimension.CumulVar(index).SetRange(0, data['vehicle_max_distance'][vehicle_id])#设置每辆车的行驶距离


In [ ]:
max_distances = [distance1, distance2, distance3, distance4]  # 每辆车的最大行驶距离

# 添加距离回调函数
def distance_callback(from_index, to_index):
    # 计算并返回距离
    ...

# 注册距离回调函数
distance_callback_index = routing.RegisterTransitCallback(distance_callback)

# 添加距离维度，也可以每辆车一个最大距离
routing.AddDimension(
    distance_callback_index,
    0,  # 不允许距离的弹性（slack）
    max_distance,  # 允许的最大行驶距离
    True,  # 开始累积值为0
    "Distance"
)

distance_dimension = routing.GetDimensionOrDie("Distance")
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    distance_dimension.CumulVar(index).SetRange(0, max_distances[vehicle_id])